导入csv数据

In [80]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

filename = 'titanic-data.csv'
df = pd.read_csv(filename)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


提出问题:有哪些因素会让船上的人生还率更高？

In [81]:
# 单一分析:
# 生存与客舱等级的关系
p_df=df[['Pclass','Survived']]
p_df.groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [82]:
# 生存与性别的关系
s_df=df[["Sex", "Survived"]]
s_df.groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [83]:
#生存与是否有家人同伴的关系
df['Is_family']=df['SibSp']+df['Parch']
f_df=df[['Is_family','Survived']]

def num_1(x):
    if x>0:
        return 1
    else:
        return 0

f_df['Is_family'] = f_df['Is_family'].apply(num_1)
f_df.groupby(['Is_family'], as_index=False).mean().sort_values(by='Survived', ascending=False)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Is_family,Survived
1,1,0.505650
0,0,0.303538


补全年龄数据

In [85]:
mean_age = df["Age"].mean()
std_age = df["Age"].std()
count_nan_age = df["Age"].isnull().sum()

rand_1 = np.random.randint(mean_age - std_age, mean_age + std_age, size = count_nan_age)
df["Age"][np.isnan(df["Age"])] = rand_1
average_age = df[["Age", "Survived"]].groupby(['Age'],as_index=False).mean()

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
